In [1]:
import pandas as pd
import geopandas as gpd 


In [25]:
parking=pd.read_csv('./parking_list.csv',encoding='utf-8',index_col=0) 

In [26]:
parking

,parking_nm,addr,Latitude,Longitude
0,신선동,부산광역시 영도구 하나길 472,35.087205,129.046782
1,봉래시장,부산광역시 영도구 대교로 9,35.092984,129.043804
2,유정아파트,부산광역시 영도구 장미계단길 123,35.093309,129.054346
3,청학시장,부산광역시 영도구 청학서로 88,35.096842,129.059161
4,청학하나,부산광역시 영도구 청학서로 88,35.096842,129.059161
...,...,...,...,...
451,녹산산단공영주차장,부산광역시 강서구 녹산산단262로50번길 53,35.083973,128.840152
452,발렛주차장,부산광역시 강서구 공항앞길13번길 37,35.178695,128.955133
453,차차주차장,부산광역시 강서구 공항앞길13번길 8,35.179754,128.954122
454,동원주차장,부산광역시 강서구 공항앞길33번길 12,35.177269,128.954308


In [40]:
busan_travel=pd.read_csv('./busan_spot.csv',encoding='utf-8') # 관광지정보read
busan_travel

,name,lat,lon
0,[부산 갈맷길 2코스] 2구간 민락교~오륙도,35.161826,129.132524
1,168계단 / 모노레일,35.117469,129.035409
2,25의용단,35.171540,129.113594
3,40계단 문화관,35.105016,129.034531
4,40계단 문화관광테마거리,35.104844,129.037585
...,...,...,...
411,AREA6,35.092801,129.043143
412,BIFF 광장,35.098833,129.029209
413,F1963,35.176991,129.115461
414,KT&G 상상마당 부산,35.154284,129.057316


In [41]:
busan_parking_gdf = gpd.GeoDataFrame(parking,geometry=gpd.points_from_xy(parking.Longitude,parking.Latitude),crs='epsg:4326')  # WGS84(EPSG:4326) :경도위도좌표계 
busan_travel_gdf = gpd.GeoDataFrame(busan_travel,geometry=gpd.points_from_xy(busan_travel.lon,busan_travel.lat),crs='epsg:4326')  

# 부산버스지오df 

In [42]:
busan_travel_gdf

,name,lat,lon,geometry
0,[부산 갈맷길 2코스] 2구간 민락교~오륙도,35.161826,129.132524,POINT (129.13252 35.16183)
1,168계단 / 모노레일,35.117469,129.035409,POINT (129.03541 35.11747)
2,25의용단,35.171540,129.113594,POINT (129.11359 35.17154)
3,40계단 문화관,35.105016,129.034531,POINT (129.03453 35.10502)
4,40계단 문화관광테마거리,35.104844,129.037585,POINT (129.03758 35.10484)
...,...,...,...,...
411,AREA6,35.092801,129.043143,POINT (129.04314 35.09280)
412,BIFF 광장,35.098833,129.029209,POINT (129.02921 35.09883)
413,F1963,35.176991,129.115461,POINT (129.11546 35.17699)
414,KT&G 상상마당 부산,35.154284,129.057316,POINT (129.05732 35.15428)


In [43]:
parking_distances = busan_parking_gdf.to_crs(epsg=5186).geometry.distance(busan_travel_gdf.to_crs(epsg=5186).iloc[0].geometry)

In [44]:
parking_distances

0      11390.081181
1      11128.091800
2      10423.779246
3       9837.448934
4       9837.448934
           ...     
451    28026.168115
452    16275.034850
453    16380.311920
454    16332.535269
455    16673.998864
Length: 456, dtype: float64

In [45]:
ts_geos = busan_travel_gdf.to_crs(epsg=5186).geometry # 관광지 좌표 리스트
x_geos = busan_parking_gdf.to_crs(epsg=5186).geometry # [장소] 좌표 리스트
distances = x_geos.distance(ts_geos[0]) # a관광지와 장소리스트 좌표 간의 거리 리스트(type : Series)
minidx = distances.idxmin() # Series에서 최소값을 가진 인덱스 return
nearest_name = parking.iloc[minidx, 0] # 인덱스를 통해 가까운 장소명 반환
nearest_dist = round(distances.iloc[minidx], 1) # 인덱스를 통해 가까운 거리값 반환(1m 단위)

In [46]:
def find_nearest(ts_point):
    distances =  x_geos.distance(ts_point)
    minidx = distances.idxmin()
    nearest_name = parking.iloc[minidx][0]
    nearest_dist = round(distances.iloc[minidx], 1)
    return nearest_name, nearest_dist 

In [49]:
busan_travel_gdf['x_nm'] = ts_geos.apply(lambda x: find_nearest(x)[0])
busan_travel_gdf['x_dist'] = ts_geos.apply(lambda x: find_nearest(x)[1])

In [52]:
busan_travel_gdf.to_clipboard()

In [51]:
busan_travel_gdf.to_csv('./busan_parking.csv',encoding='utf-8',index=False)